In [1]:
import os
import cv2
import torch

from face_recognition.utils.data_augmentation import augment_and_normalize

In [2]:
root = os.path.join('face_recognition', 'data', 'images', 'lfw_aligned')
tmp = os.listdir(root)[0]
folder = os.path.join(root, tmp)
folder, os.listdir(folder)[0]

('face_recognition\\data\\images\\lfw_aligned\\Aaron_Eckhart',
 'Aaron_Eckhart_0001.jpg')

In [3]:
pth = '.\\face_recognition\\data\\images\\lfw_aligned\\Aaron_Eckhart\\Aaron_Eckhart_0001.jpg'

img = cv2.imread(pth)

In [4]:
tens = torch.from_numpy(img).unsqueeze(0).float()

In [5]:
augmented = augment_and_normalize(tens)

In [6]:
augmented[0].shape

torch.Size([1, 3, 224, 224])

In [ ]:
hparams = {'margin': 0.02, 'optimizer': 'adagrad', 'weight_decay': 1e-5, 'lr': 0.05}
backbone = FaceNetResnet(pretrained=True)
model = LightningFaceNet(hparams, backbone)

In [ ]:
model = LightningFaceNet.load_from_checkpoint(checkpoint, hparams=hparams, model=backbone)

In [ ]:
model

In [ ]:
trainer = pl.Trainer(
    gpus=1 if torch.cuda.is_available() else 0,
    max_epochs=3,
)

In [ ]:
from torch.utils.data import DataLoader

test_loader = DataLoader(lfw_set, batch_size=16, shuffle=True, num_workers=8)
trainer.test(model, test_loader)

In [ ]:
model_name = os.path.join(results_path, 'presentation_model.pth')

In [ ]:
torch.save(model.model.state_dict(), model_name)

In [ ]:
from face_recognition.models.FaceNet import load_weights

In [ ]:
model = load_weights(model_name)


In [ ]:
model

In [ ]:
enc = model(lfw_set[0][0].unsqueeze(0))

In [ ]:
enc.shape